## Enyzme Design with Pyrosetta

Thanks to the new version of the PyRosetta bindings (nice work Sergey!) most of the enzdes code now works in python. There are a few things to note along the way. This demo will show you how to run enzyme design in PyRosetta using the files located at https://github.com/dacarlin/bagel-foldit. That location has a PDB file, an enzyme design file, params file (and rotamers)

Note: Try "git clone https://github.com/dacarlin/bagel-foldit" in the same directory that you have this notebook. Also, you must have already downloaded and pip installed pyrosetta4 bindings to your python. This assumes you have Anaconda's python 2.7 installed as well.

You can always pass your extra desired options into the pyrosetta.init string

This is not entirely reproducing the xml as of yet.

In [1]:
import rosetta
import pyrosetta
pyrosetta.init('-extra_res_fa ./bagel-foldit/cid92930.params -run:preserve_header T')

Found rosetta database at: /home/steve/anaconda2/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:7ce9d5bf5174d497560ce1624761e0c166dc30c3 2016-07-22 16:07:06 -0700] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [2]:
p = pyrosetta.pose_from_file('./bagel-foldit/bagel.pdb')

In [3]:
# This must be turned on globally for the enable_cst_scorefunction to turn on the fnr term (which is actually res_type_constraint, see protocols:protein_design_interface:design_utils)
# and also see protocols.enzdes.endes_utils
# Note, how we can print the value before and after it's set to verify our changes

print pyrosetta.rosetta.basic.options.get_real_option('enzdes:favor_native_res')
pyrosetta.rosetta.basic.options.set_real_option('enzdes:favor_native_res',2.0)

print pyrosetta.rosetta.basic.options.get_real_option('enzdes:lig_packer_weight')
pyrosetta.rosetta.basic.options.set_real_option('enzdes:lig_packer_weight',1.5)
#print pyrosetta.rosetta.basic.options.get_real_option('enzdes:lig_packer_weight')

## These must be set globally for the detect_ligand_interace to work, there should be getters and setters, but 
## as of today, there aren't any
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut1')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut2')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut3')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut4')
print pyrosetta.rosetta.basic.options.get_boolean_option('enzdes:detect_design_interface')

pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut1',8.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut2',10.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut3',12.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut4',14.0)
pyrosetta.rosetta.basic.options.set_boolean_option('enzdes:detect_design_interface',True)

#print pyrosetta.rosetta.basic.options.get_real_option('enzdes:lig_packer_weight')
#print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut1')

0.5
1.0
0.0
0.0
10.0
10.0
False


In [4]:
# Optional, if you want to watch in Pymol
pm = rosetta.protocols.moves.PyMolMover()
pm.keep_history(True)
pm.apply(p) ## Note, this started the Pymol counter to state 1 (view at bottom right hand corner)

In [5]:
## Setup Scorefunctions

# get a generic scorefxn
sfxn = pyrosetta.get_fa_scorefxn()

# This turns on certain scorefunction terms associated with constraints (but not all of them!)
rosetta.protocols.enzdes.enzutil.enable_constraint_scoreterms(sfxn)

# Also, get a soft_rep score function, here the weights for fa_rep are turned down so that
# when new residues are designed in, if they are clashing before repacking, it won't be immediately
# discarded as bad in energy.
soft_rep = rosetta.core.scoring.ScoreFunctionFactory.create_score_function("soft_rep")

In [6]:
## Setup movers!

# Create the AddorRemoveMatchCsts object, read in the file, and apply/add them to the pose
addcsts = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
addcsts.cstfile('bagel-foldit/cid92930.enzdes.cst')
addcsts.set_cst_action( rosetta.protocols.enzdes.CstAction.ADD_NEW )
addcsts.apply(p)

# This is the enzdes generic move-the-ligand-around sampling mover
predock = rosetta.protocols.enzdes.PredesignPerturbMover()
predock.trans_magnitude(0.1)
predock.rot_magnitude(2)
## This set ligand must be set to the ligand, here it is hard coded for the example
predock.set_ligand(446) # or p.n_total() Assumes the ligand is the last

## These two objects/movers are the business end of enzdes in rosetta. They setup the object, some options,
## with the only difference here being the second one turns backbone (bb) minimziations on
enzdes = rosetta.protocols.enzdes.EnzRepackMinimize()
enzdes.set_scorefxn_minimize(sfxn)
enzdes.set_scorefxn_repack(soft_rep)
enzdes.set_min_lig(True)
enzdes.set_min_rb(True)
enzdes.set_min_sc(True)
enzdes.set_design(True)

enzdes_wbb = rosetta.protocols.enzdes.EnzRepackMinimize()
enzdes_wbb.set_scorefxn_minimize(sfxn)
enzdes_wbb.set_min_lig(True)
enzdes_wbb.set_min_rb(True)
enzdes_wbb.set_min_sc(True)
enzdes_wbb.set_design(True)
enzdes_wbb.set_scorefxn_repack(soft_rep)
enzdes_wbb.set_min_bb(True)

## Notes for the future

# min in stages?  # further investigation required here
# backrub? # further investigation required here

# Now score the protein (this includes the constraints in the score)
sfxn(p) # you need this for nbr graph to populate

-650.87487918439

In [7]:
# Setup task opertations and add them to the enzdes movers

tf = rosetta.core.pack.task.TaskFactory()

# Sets up the packing/design shells from the global options set above
dp = rosetta.protocols.enzdes.DetectProteinLigandInterface()
dp.init_from_options()

## Applies the enigmatic Task Operation LimitAromaticChi2 operation to the packer task.
## This is one of those legacy operations that you have to ask the Rosetta elders to explain
## Rosetta source credits Nobuyasu for this TO
limchi2 = rosetta.protocols.toolbox.task_operations.LimitAromaChi2Operation()

# This restricts the residues define in the constraint file to only be allowed to pack, not designable
canttouchcatres = rosetta.protocols.enzdes.SetCatalyticResPackBehavior()
canttouchcatres.set_fix_catalytic_aa(False) ## seems to freeze them, no repacking if set to True

# Push back all of our TO's onto the task factory (see Design Patterns on Factorys)
tf.push_back(dp)
tf.push_back(canttouchcatres)
tf.push_back(limchi2)

# Create a packer task, specifically for the cstopt mover to work
pt = tf.create_task_and_apply_taskoperations(p)

enzdes.task_factory( tf ) # sets the enzdes movers to use our task factory set with our task operations 
enzdes_wbb.task_factory( tf )

In [8]:
# Just leaving this here as an example, but this allows for manual inspection of your packer task 
# after all of the task operations have been applied. You could also apply one at a time to verify how
# they are working and that they are indeed working.

#for i in xrange(1,p.total_residue()+1):
#    print i,pt.being_designed(i),pt.being_packed(i)

In [9]:
# Warning, what follows is a bunch of bs all because there is not a getter/setter
# for cst_opt in the enzdesmovers

# Mover sub-classing -----------------------------------                                                         
# Thank you Sergey!
#
class My_New_Mover(rosetta.protocols.moves.Mover):
    def __init__(self,sfxn,pt):
        print( 'My_New_Mover.__init__...' )
        rosetta.protocols.moves.Mover.__init__(self)
        self.sfxn = sfxn
        self.pt = pt
        
    def get_name(self): return 'My_New_Mover'

    def apply(self, p):
        #print( 'My_New_Mover.apply:', type(p) )
        #print( 'This My_New_Mover apply...' )

        cstopt = rosetta.protocols.enzdes.EnzdesBaseProtocol()
        cstopt.set_scorefxn( self.sfxn )
        cstopt.set_minimize_options(True, False, True, True) # check this fn signature for details
        
        # this actuall runs the minimizer !
        cstopt.cst_minimize(p, self.pt, True)

cstopt = My_New_Mover(sfxn,pt)

## Note that a mover must have an apply function that takes a pose and does something to it
## in this case, cst opt derives from Enzdesbase protocol, but in the xml it's call from an option in
## the enzdesrepackmin mover... blah blah blah, long story short, this ends up working in pyrosetta since
## there are currently no setters for this option in the enzrepackmin mover... this needs to be fixed

My_New_Mover.__init__...


In [10]:
# just because we can, lets make a mover that will dump our designed pdb after running the generic monte carlo

In [11]:
## Now we string all of our movers together so that one mover can be fed into the generic monte carlo

parsed = rosetta.protocols.moves.SequenceMover()
parsed.add_mover(pm) # state 2, nothing should have changed
parsed.add_mover(predock)
parsed.add_mover(pm)  # state 3, only the ligand should have moved
parsed.add_mover(cstopt)
parsed.add_mover(pm)  # state 4 , the constraints should be optimized/satisfied
parsed.add_mover(enzdes)
parsed.add_mover(pm) # state 5, designing and packing within the shells
parsed.add_mover(enzdes_wbb)
parsed.add_mover(pm) # state 6, now, the backbone might have minor perturbations (sometimes barely noticeable)
## These states track with running
## a simulation for testing using "mc.apply(p)"

In [12]:
## now we setup the Generic Monte Carlo object to run the simulation, this will run our sequence mover/parsedprotocol
## for X cycles, you can set the criterion (sfxn) and filters and other stuff

# The XML line looks like this
##<GenericMonteCarlo name=multi_dd mover_name=dock_des filter_name=allcst trials=10 sample_type=low temperature=0.6 drift=1\/>

mc = rosetta.protocols.simple_moves.GenericMonteCarloMover()
mc.set_drift(True) # this sets drift for the maxtrials (not technically mc anymore)
mc.set_maxtrials(1)  # CHANGE THIS TO 10 for real runs
mc.set_sampletype('low')
mc.set_temperature(0.6)
mc.set_mover(parsed)
mc.set_scorefxn(sfxn)
#mc.apply(p)

In [13]:
# This is how you would normally run for nstruct 10
# Note: This is not actually necessary depending on your goals, you can
# also mc.apply(p), or store the energies over the simulation, or p.dump_pdb() etc.
nstruct = 2
job_output = 'test_output'
jd = pyrosetta.PyJobDistributor(job_output, nstruct, sfxn)
temp_pose = rosetta.core.pose.Pose()    # a temporary pose to export to PyMOL                                                                     
temp_pose.assign(p)
counter = 0    # for pretty output to PyMOL                                                                                     

while not jd.job_complete:
    
    counter += 1
    
    test_pose = rosetta.core.pose.Pose()
    # set staring pose to input pose
    test_pose.assign(p)    
    # change pose name for pretty viewing in PyMol
    test_pose.pdb_info().name(job_output + '_' + str(counter))

    # apply mc mover (with all of the stuff)
    mc.apply(test_pose)
    
    # have the jd output the resulting low energy model
    jd.output_decoy(test_pose)

